In [1]:
from group import DummyGroup
from utils import log_2, pow_2
from unipolynomial import UniPolynomial
from mle2 import MLEPolynomial
from kzg10 import Commitment, KZG10Commitment

In [2]:
class ProofTranscript:
    def __init__(self, field):
        self.field = field
        self.state = 0  # Initial state
    
    def absorb(self, element):
        """
        Absorb a field element into the transcript.
        
        Args:
            element: A field element to be absorbed. 
        """
        if isinstance(element, int):
            element = self.field(element)
        elif isinstance(element, sage.rings.integer.Integer):
            element = self.field(element)
        elif isinstance(element, Commitment):
            element = element.value
        elif isinstance(element, list):
            element = sum(element)
        elif not element.parent() == self.field:
            raise TypeError("Must absorb a field element")
        
        # if not isinstance(element, self.field.Element):
        #     raise TypeError("Must absorb a field element")
        
        # Simple hash function: multiply by a prime and add the element
        bs = str(element).encode('utf-8')
        n = int.from_bytes(bs, 'big')
        self.state = (self.state * int(31) + n) % self.field.order()
    
    def squeeze(self):
        """
        Generate a random field element based on the current state.
        
        Returns:
            A field element.
        """
        # Use the current state to generate a "random" field element
        self.state = (self.state * 31 + 17) % self.field.order()
        return self.field(self.state)
    
    def clone(self):
        new = ProofTranscript(self.field)
        new.state = self.state
        return new


In [3]:
F193.<a> = GF(193)
R193.<X> = F193[]
R193
Fp=F193
Fp2 = Fp.extension(2, 'b')
Fp, Fp2

(Finite Field of size 193, Finite Field in b of size 193^2)

In [4]:
UniPolynomial.set_scalar(Fp(0), Fp)
Commitment.set_scalar(Fp(0))

In [5]:
R_2.<X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2> = Fp[]; R_2

Multivariate Polynomial Ring in X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, Y0, Y1, Y2, Y3, Y4, Y5, Y6, Y7, A0, A1, A2, B0, B1, B2 over Finite Field of size 193

In [6]:
omega = Fp.primitive_element()^3; omega, omega^64 == 1, omega^32 == -1

(125, True, True)

In [7]:
H16 = [(omega^4)^i for i in range(16)]
H8 = [(omega^8)^i for i in range(8)]
H4 = [(omega^16)^i for i in range(4)]

In [8]:
G1 = DummyGroup(Fp)
G2 = DummyGroup(Fp)

In [9]:
mle = MLEPolynomial([1, 4, 2, 6], 2)
point = [3, 3]  # Replace with actual Scalar values

print("point=", point)
quo, rem = mle.decompose_by_div(point)

print(f"Number of quotients: {len(quo)}")

for i, q in enumerate(quo):
    # q * MLEPolynomial.from_coeffs([-point[i], 1])
    print(f"q_{i}: num_var={q.num_var}, evals={q.evals}")
print("rem=", rem)
rem == mle.evaluate(point)

point= [3, 3]
Number of quotients: 2
q_0: num_var=0, evals=[6]
q_1: num_var=1, evals=[1, 2]
rem= 22


True

In [10]:
6 * (X0-3) + (X0 + 1) * (X1-3) + 22

X0*X1 + 3*X0 + X1 + 1

In [11]:
MLEPolynomial.compute_coeffs_from_evals([1, 4, 2, 6])

[1, 3, 1, 1]

In [12]:
q_1 = MLEPolynomial.compute_coeffs_from_evals([1, 2]); q_1

[1, 1]

In [13]:
Basis = [1, X0, X1, X0 * X1, X2, X0 * X2, X1 * X2, X0 * X1 * X2]
def poly(coeffs):
    return sum([coeff * basis for coeff, basis in zip(coeffs, Basis)])

poly([1, 4, 2, 6])

6*X0*X1 + 4*X0 + 2*X1 + 1

In [14]:
6 * (X0-3) + (X0 + 1) * (X1-3) + 22

X0*X1 + 3*X0 + X1 + 1

In [15]:
# def ntt_core(vs, twiddle):
#     n = len(vs)
#     k = log_2(n)
#     half = 1
#     for i in range(k):
#         for j in range(0, n, 2*half):
#             for l in range(j, j+half):
#                 vs[l+half] = vs[l+half] + twiddle * vs[l]
#         half <<= 1
#     return vs

# def compute_evals_from_coeffs(f_coeffs):
#     """
#     Compute the evaluations of the polynomial from the coefficients.
#         Time: O(n * log(n))
#     """
#     return ntt_core(f_coeffs, 1)

# def compute_coeffs_from_evals(f_evals):
#     """
#     Compute the evaluations of the polynomial from the coefficients.
#         Time: O(n * log(n))
#     """
#     return ntt_core(f_evals, -1)


In [16]:
def periodic_poly(dim, degree):
    """
    Compute the periodic polynomial Phi(X^d)

        Phi_k(X)   = 1 + X   + X^2  + X^3  + ... + X^(2^k-1)
        Phi_k(X^d) = 1 + X^d + X^2d + X^3d + ... + X^(2^(k-1))d

    Args:
        dim: dimension of the space of size 2^k
        degree: degree of X^d

    Returns:
        list: the coefficients of Phi(X^d)
    """
    n = pow_2(dim)
    coeffs = [0] * (n * degree)
    for i in range(n):
        coeffs[i * degree] = 1
    return UniPolynomial(coeffs)


In [17]:
kzg10 = KZG10Commitment(G1, G2, 24); 

In [18]:
kzg10.setup(secret_symbol=X0, g1_generator=1, g2_generator=1)
kzg10.srs, kzg10.srs2

([1,
  X0,
  X0^2,
  X0^3,
  X0^4,
  X0^5,
  X0^6,
  X0^7,
  X0^8,
  X0^9,
  X0^10,
  X0^11,
  X0^12,
  X0^13,
  X0^14,
  X0^15,
  X0^16,
  X0^17,
  X0^18,
  X0^19,
  X0^20,
  X0^21,
  X0^22,
  X0^23,
  X0^24],
 [1,
  X0,
  X0^2,
  X0^3,
  X0^4,
  X0^5,
  X0^6,
  X0^7,
  X0^8,
  X0^9,
  X0^10,
  X0^11,
  X0^12,
  X0^13,
  X0^14,
  X0^15,
  X0^16,
  X0^17,
  X0^18,
  X0^19,
  X0^20,
  X0^21,
  X0^22,
  X0^23,
  X0^24])

In [19]:
ff = [1, 4, 2, 6]
ff_cm = kzg10.commit(ff)
deg_arg = kzg10.prove_degree_bound(ff_cm, ff, 4)
deg_arg

(Commitment(6*X0^24 + 2*X0^23 + 4*X0^22 + X0^21), 21)

In [20]:
kzg10.verify_degree_bound(ff_cm, deg_arg, 4)

True

In [21]:
def prove_zeromorph_naive(C, f, k, point, v, transcript, debug=0):  
    """

        r(X) = ⟦f_mle⟧_n - v * 𝚽_n(z) - ∑_k(z^{2^k} * 𝚽_{n-k-1}(z^{2^{k+1}}) - uk * 𝚽_{n-k}(z^{2^k})) * ⟦q_k⟧_k
             = f(X) - v * 𝚽_n(z) 
                - ∑_k (z^{2^k} * 𝚽_{n-k-1}(z^{2^{k+1}}) - uk * 𝚽_{n-k}(z^{2^k})) * q_k(X)
    f: polynomial
    k: number of variables
    point: (u0, u1, ..., u_{k-1})
    v: evaluation value
    transcript: transcript
    """
    n = len(f)
    k = len(point)
    evals = f.copy()
    print(f"P> f={f}, point={point}, v={v}")

    quotients, rem = MLEPolynomial(f, k).decompose_by_div(point)
    print(f"quotients={quotients}, rem={rem}")
    assert rem == v, "Evaluation does not match"
    f_uni = UniPolynomial(f)

    print(f"P> ▶️▶️ C={C}")
    print(f"P> ▶️▶️ point={point}")
    print(f"P> ▶️▶️ v={v}")
    transcript.absorb(C)
    transcript.absorb(point)
    transcript.absorb(v)

    q_cm_vec = []
    q_uni_vec = []
    deg_arg_vec = []

    for i in range(k):
        qi = quotients[i]
        qi_cm = kzg10.commit(qi.evals)
        q_cm_vec.append(qi_cm)
        q_uni_vec.append(UniPolynomial(qi.evals))
        print(f"P> ▶️▶️ q_cm={qi_cm}")
        transcript.absorb(qi_cm)
    
    print(f"P> f_cm={C}, q_cm_vec={q_cm_vec}, q_uni_vec={q_uni_vec}")

    for i in range(k):
        qi_deg_arg = kzg10.prove_degree_bound(q_cm_vec[i], q_uni_vec[i].coeffs, pow_2(i))
        deg_arg_vec.append(qi_deg_arg)
        print(f"P> ▶️▶️ deg_arg={qi_deg_arg}")
        transcript.absorb(qi_deg_arg[0])
        transcript.absorb(qi_deg_arg[1])

    f_deg_arg = kzg10.prove_degree_bound(C, f_uni.coeffs, pow_2(k))
    deg_arg_vec.append(f_deg_arg)
    print(f"P> ▶️▶️ deg_arg={f_deg_arg}")
    transcript.absorb(f_deg_arg[0])
    transcript.absorb(f_deg_arg[1])

    zeta = transcript.squeeze()
    if debug > 0:
        print(f"P> zeta={zeta}")

    # compute r(X) = f(X) - v * phi_n(zeta) - ∑_i (c_i * qi(X))
    phi_uni_at_zeta = periodic_poly(k, 1).evaluate(zeta)
    if debug > 1:
        print(f"P> f_uni={f_uni}, v={v}, phi_uni_at_zeta={phi_uni_at_zeta}")

    r_uni = f_uni - UniPolynomial([phi_uni_at_zeta * v])
    for i in range(k):
        c_i = zeta^(pow_2(i)) * periodic_poly(k-i-1, pow_2(i+1)).evaluate(zeta) \
                   - point[i] * periodic_poly(k-i, pow_2(i)).evaluate(zeta)
        r_uni -= Fp(c_i) * q_uni_vec[i]

    if debug > 1:
        print("P> r_uni=", r_uni)
    if debug > 0:
        assert r_uni.evaluate(zeta) == 0, f"Evaluation does not match, {r_uni.evaluate(zeta)}!=0"
        print(f"P> 👀 r(zeta={zeta}) == 0 ✅")
    
    r_at_0_arg = kzg10.prove_eval(r_uni.coeffs, zeta, 0)

    if debug > 1:
        v_cm = kzg10.commit(UniPolynomial([phi_uni_at_zeta * v]).coeffs)
        r_cm = C - v_cm 
        print("P> C - v_cm=", r_cm)
        for i in range(k):
            c_i = zeta^(pow_2(i)) * periodic_poly(k-i-1, pow_2(i+1)).evaluate(zeta) \
                       - point[i] * periodic_poly(k-i, pow_2(i)).evaluate(zeta)
            r_cm = r_cm - q_cm_vec[i] * c_i

        print(f"P> r_cm={r_cm}, r({zeta})={r_uni.evaluate(zeta)}")
        checked = kzg10.verify_eval(r_cm, r_at_0_arg, zeta, 0)
        print("P> r_at_0_arg=", r_at_0_arg)
        print("P> 👀  r(zeta) == 0 ✅" if checked else "👀  r(zeta) == 0 ❌")

    return (q_cm_vec, deg_arg_vec, r_at_0_arg)

In [22]:
def verify_zeromorph_naive(f_cm, num_var, point, v, prf, transcript):
    """
    Verify an evaluation proof(argument) of the MLE polynomial in Zeromorph+KZG10.

    Args:
        f_cm: commitment to the MLE polynomial
        num_var: number of variables
        point: (u0, u1, ..., u_{k-1})
        v: evaluation value
        prf: proof
    
    Returns:
        bool: True if the proof is valid, False otherwise.
    """
    n = pow_2(num_var)
    k = len(point)
    assert k == num_var, "Number of variables must match the point"

    print(f"V> ▶️▶️ f_cm={f_cm}")
    print(f"V> ▶️▶️ point={point}")
    print(f"V> ▶️▶️ v={v}")
    transcript.absorb(f_cm)
    transcript.absorb(point)
    transcript.absorb(v)
    
    # decompose the proof
    q_cm_vec, deg_arg_vec, r_at_0_arg = prf
    
    for q in q_cm_vec:
        print(f"V> ▶️▶️ q_cm={q}")
        transcript.absorb(q)

    q_cm_vec.append(f_cm)
    for i in range(k+1):
        # TODO: add support for degree_bound proofs
        p = deg_arg_vec[i]
        assert kzg10.verify_degree_bound(q_cm_vec[i], p, pow_2(i))
        print(f"V> ▶️▶️ deg_arg={p}")
        transcript.absorb(p[0])
        transcript.absorb(p[1])

    zeta = transcript.squeeze()
    print(f"V> zeta={zeta}")
    
    phi_uni_at_zeta = periodic_poly(k, 1).evaluate(zeta)
    v_cm = kzg10.commit(UniPolynomial([phi_uni_at_zeta * v]).coeffs)
    r_cm = f_cm - v_cm 
    for i in range(k):
        c_i = zeta^(pow_2(i)) * periodic_poly(k-i-1, pow_2(i+1)).evaluate(zeta) \
                   - point[i] * periodic_poly(k-i, pow_2(i)).evaluate(zeta)
        r_cm = r_cm - q_cm_vec[i] * c_i
    print(f"V> r_cm={r_cm}")
    checked = kzg10.verify_eval(r_cm, r_at_0_arg, zeta, 0)
    print("V> r_at_0_arg=", r_at_0_arg)
    print("V> 👀  r(zeta) == 0 ✅" if checked else "👀  r(zeta) == 0 ❌")

    return checked

In [23]:
transcript = ProofTranscript(Fp)

f_uni = UniPolynomial([1, 4])
f_cm = kzg10.commit(f_uni.coeffs)

prf = prove_zeromorph_naive(f_cm, [1, 4], 1, [3], 10, transcript.clone(), debug=2)
verify_zeromorph_naive(f_cm, 1, [3], 10, prf, transcript.clone())

P> f=[1, 4], point=[3], v=10
quotients=[MLEPolynomial([3], 0)], rem=10
P> ▶️▶️ C=Commitment(4*X0 + 1)
P> ▶️▶️ point=[3]
P> ▶️▶️ v=10
P> ▶️▶️ q_cm=Commitment(3)
P> f_cm=Commitment(4*X0 + 1), q_cm_vec=[Commitment(3)], q_uni_vec=[3]
P> ▶️▶️ deg_arg=(Commitment(3*X0^24), 24)
P> ▶️▶️ deg_arg=(Commitment(4*X0^24 + X0^23), 23)
P> zeta=47
P> f_uni=1 + 4x, v=10, phi_uni_at_zeta=48
P> r_uni= 5 + 4x
P> 👀 r(zeta=47) == 0 ✅
P> C - v_cm= Commitment(4*X0 - 93)
P> r_cm=Commitment(4*X0 + 5), r(47)=0
P> r_at_0_arg= Commitment(4)
P> 👀  r(zeta) == 0 ✅
V> ▶️▶️ f_cm=Commitment(4*X0 + 1)
V> ▶️▶️ point=[3]
V> ▶️▶️ v=10
V> ▶️▶️ q_cm=Commitment(3)
V> ▶️▶️ deg_arg=(Commitment(3*X0^24), 24)
V> ▶️▶️ deg_arg=(Commitment(4*X0^24 + X0^23), 23)
V> zeta=47
V> r_cm=Commitment(4*X0 + 5)
V> r_at_0_arg= Commitment(4)
V> 👀  r(zeta) == 0 ✅


True

In [24]:
isinstance(Fp(3), (type(Fp(2))))

True

In [25]:
transcript = ProofTranscript(Fp)

f_uni = UniPolynomial([1, 4, 2, 6])
f_cm = kzg10.commit(f_uni.coeffs)

prf = prove_zeromorph_naive(f_cm, [1, 4, 2, 6], 2, [3, 3], 22, transcript.clone(), debug=2)
verify_zeromorph_naive(f_cm, 2, [3, 3], 22, prf, transcript.clone())

P> f=[1, 4, 2, 6], point=[3, 3], v=22
quotients=[MLEPolynomial([6], 0), MLEPolynomial([1, 2], 1)], rem=22
P> ▶️▶️ C=Commitment(6*X0^3 + 2*X0^2 + 4*X0 + 1)
P> ▶️▶️ point=[3, 3]
P> ▶️▶️ v=22
P> ▶️▶️ q_cm=Commitment(6)
P> ▶️▶️ q_cm=Commitment(2*X0 + 1)
P> f_cm=Commitment(6*X0^3 + 2*X0^2 + 4*X0 + 1), q_cm_vec=[Commitment(6), Commitment(2*X0 + 1)], q_uni_vec=[6, 1 + 2x]
P> ▶️▶️ deg_arg=(Commitment(6*X0^24), 24)
P> ▶️▶️ deg_arg=(Commitment(2*X0^24 + X0^23), 23)
P> ▶️▶️ deg_arg=(Commitment(6*X0^24 + 2*X0^23 + 4*X0^22 + X0^21), 21)
P> zeta=70
P> f_uni=1 + 4x + 2x^2 + 6x^3, v=22, phi_uni_at_zeta=185
P> r_uni= 111 + 117x + 2x^2 + 6x^3
P> 👀 r(zeta=70) == 0 ✅
P> C - v_cm= Commitment(6*X0^3 + 2*X0^2 + 4*X0 - 16)
P> r_cm=Commitment(6*X0^3 + 2*X0^2 - 76*X0 - 82), r(70)=0
P> r_at_0_arg= Commitment(6*X0^2 + 36*X0 - 65)
P> 👀  r(zeta) == 0 ✅
V> ▶️▶️ f_cm=Commitment(6*X0^3 + 2*X0^2 + 4*X0 + 1)
V> ▶️▶️ point=[3, 3]
V> ▶️▶️ v=22
V> ▶️▶️ q_cm=Commitment(6)
V> ▶️▶️ q_cm=Commitment(2*X0 + 1)
V> ▶️▶️ deg_arg=(C

True

In [26]:
def prove_zeromorph_opt(f_cm, f, k, point, v, transcript, debug=0):  
    """
    Optimized version of prove_zeromorph_naive

        - Degree bound arguments are aggregated and merged into the evaluation proof

        r(X) = ⟦f_mle⟧_n - v * 𝚽_n(z) - ∑_k(z^{2^k} * 𝚽_{n-k-1}(z^{2^{k+1}}) - uk * 𝚽_{n-k}(z^{2^k})) * ⟦q_k⟧_k
             = f(X) - v * 𝚽_n(z) 
                - ∑_k (z^{2^k} * 𝚽_{n-k-1}(z^{2^{k+1}}) - uk * 𝚽_{n-k}(z^{2^k})) * q_k(X)

    f: polynomial
    k: number of variables
    point: (u0, u1, ..., u_{k-1})
    v: evaluation value
    transcript: transcript
    """
    n = len(f)
    k = len(point)
    evals = f.copy()
    print(f"P> f={f}, point={point}, v={v}")

    # Round 1.

    quotients, rem = MLEPolynomial(f, k).decompose_by_div(point)
    print(f"quotients={quotients}, rem={rem}")
    assert rem == v, "Evaluation does not match"
    f_uni = UniPolynomial(f)

    print(f"P> ▶️▶️ f_cm={f_cm}")
    print(f"P> ▶️▶️ point={point}")
    print(f"P> ▶️▶️ v={v}")
    transcript.absorb(f_cm)
    transcript.absorb(point)
    transcript.absorb(v)

    q_cm_vec = []
    q_uni_vec = []
    deg_arg_vec = []

    for i in range(k):
        qi = quotients[i]
        qi_cm = kzg10.commit(qi.evals)
        q_cm_vec.append(qi_cm)
        q_uni_vec.append(UniPolynomial(qi.evals))
        print(f"P> ▶️▶️ q_cm={qi_cm}")
        transcript.absorb(qi_cm)
    
    print(f"P> f_cm={f_cm}, q_cm_vec={q_cm_vec}, q_uni_vec={q_uni_vec}")

    # Round 2.

    beta = transcript.squeeze()
    print(f"P> ◀️◀️ beta={beta}")

    q_hat_uni = UniPolynomial([0])
    coeffs = [0] * n
    coeffs.append(1)
    beta_power = 1
    for i in range(k):
        print(f"P> coeffs={coeffs}, coeffs[pow_2(i)]={coeffs[pow_2(i)]}")
        x_deg_2_to_i_uni = UniPolynomial(coeffs[pow_2(i):])
        print(f"P> x_deg_2_to_i_uni[{i}]={x_deg_2_to_i_uni}")
        q_hat_uni += x_deg_2_to_i_uni * q_uni_vec[i] * Fp(beta_power)
        beta_power *= beta
    print(f"P> q_hat_uni={q_hat_uni}")
    
    q_hat_cm = kzg10.commit(q_hat_uni.coeffs)
    print(f"P> ▶️▶️ q_hat_cm={q_hat_cm}")
    transcript.absorb(q_hat_cm)

    # Round 3.

    zeta = transcript.squeeze()
    print(f"P> ◀️◀️ zeta={zeta}")

    # compute r(X) = f(X) - v * phi_n(zeta) - ∑_i (c_i * qi(X))
    #
    #   r_ζ(X) = f(X) - v * 𝚽_k(ζ) 
    #            - ∑_i ( ζ^{2^i} * 𝚽_{k-i-1}(ζ^{2^{i+1}}) - u_i * 𝚽_{k-i}(ζ^{2^i}) ) * q_i(X)
    #          = f(X) - v_𝚽 - ∑_i c_i * q_i(X)
    
    phi_uni_at_zeta = periodic_poly(k, 1).evaluate(zeta)
    if debug > 1:
        print(f"P> f_uni={f_uni}, v={v}, phi_uni_at_zeta={phi_uni_at_zeta}")

    r_uni = f_uni - UniPolynomial([phi_uni_at_zeta * v])
    for i in range(k):
        c_i = zeta^(pow_2(i)) * periodic_poly(k-i-1, pow_2(i+1)).evaluate(zeta) \
                   - point[i] * periodic_poly(k-i, pow_2(i)).evaluate(zeta)
        r_uni -= c_i * q_uni_vec[i]

    if debug > 1:
        print("P> r_uni=", r_uni)
    if debug > 0:
        assert r_uni.evaluate(zeta) == 0, f"Evaluation does not match, {r_uni.evaluate(zeta)}!=0"
        print(f"P> 👀 r(zeta={zeta}) == 0 ✅")

    # compute h(X) = q_hat(X) - ∑_i ( beta^{i} * X^{2^k - 2^i} * q_i(X) ) 
    #
    #    h_ζ(X) = q_hat(X) - ∑_i ( beta^{i} * ζ^{2^k - 2^i} * q_i(X) )

    h_uni = q_hat_uni
    for i in range(k):
        e_i = (beta**i) * (zeta**(pow_2(k) - pow_2(i)))
        h_uni -= e_i * q_uni_vec[i]

    if debug > 1:
        print(f"P> h_uni={h_uni}")
    if debug > 0:
        assert h_uni.evaluate(zeta) == 0, f"Evaluation does not match, {h_uni.evaluate(zeta)}!=0"
        print(f"P> 👀 h(zeta={zeta}) == 0 ✅")
    
    alpha = transcript.squeeze()
    print(f"P> ◀️◀️ alpha={alpha}")

    a_uni = h_uni + r_uni * alpha

    if debug > 0:
        assert a_uni.evaluate(zeta) == 0, f"Evaluation does not match, {a_uni.evaluate(zeta)}!=0"
        print(f"P> 👀 a(zeta={zeta}) == 0 ✅")

    a_uni_cm = kzg10.commit(a_uni.coeffs)

    arg = kzg10.prove_eval_and_degree(a_uni_cm, a_uni.coeffs, zeta, 0, kzg10.max_degree - pow_2(k))

    # r_at_0_arg = kzg10.prove_eval(r_uni.coeffs, zeta, 0)

    # if debug > 1:
    #     v_cm = kzg10.commit(UniPolynomial([phi_uni_at_zeta * v]).coeffs)
    #     r_cm = C - v_cm 
    #     print("P> C - v_cm=", r_cm)
    #     for i in range(k):
    #         c_i = zeta^(pow_2(i)) * periodic_poly(k-i-1, pow_2(i+1)).evaluate(zeta) \
    #                    - point[i] * periodic_poly(k-i, pow_2(i)).evaluate(zeta)
    #         r_cm = r_cm - q_cm_vec[i] * c_i

    #     print(f"P> r_cm={r_cm}, r({zeta})={r_uni.evaluate(zeta)}")
    #     checked = kzg10.verify_eval(r_cm, r_at_0_arg, zeta, 0)
    #     print("P> r_at_0_arg=", r_at_0_arg)
    #     print("P> 👀  r(zeta) == 0 ✅" if checked else "👀  r(zeta) == 0 ❌")

    return (q_cm_vec, q_hat_cm, a_uni_cm, arg)

In [32]:
def verify_zeromorph_opt(f_cm, prf, k, point, v, transcript, debug=0):
    """
    Optimized version of verify_zeromorph

        - Degree bound arguments are aggregated and merged into the evaluation proof
    """
    k = len(point)
    n = pow_2(k)

    # Round 1.

    print(f"V> ⏬ f_cm={f_cm}")
    print(f"V> ⏬ point={point}")
    print(f"V> ⏬ v={v}")
    transcript.absorb(f_cm)
    transcript.absorb(point)
    transcript.absorb(v)

    q_cm_vec, q_hat_cm, a_uni_cm, eval_deg_arg = prf

    for i in range(k):
        qi_cm = q_cm_vec[i]
        print(f"V> ▶️▶️ q_cm={qi_cm}")
        transcript.absorb(qi_cm)

    # Round 2.

    beta = transcript.squeeze()
    print(f"V> ◀️◀️ beta={beta}")

    print(f"V> ▶️▶️ q_hat_cm={q_hat_cm}")
    transcript.absorb(q_hat_cm)

    # Round 3.

    zeta = transcript.squeeze()
    print(f"V> ◀️◀️ zeta={zeta}")

    # compute r(X) = f(X) - v * phi_n(zeta) - ∑_i (c_i * qi(X))
    #
    #   r_ζ(X) = f(X) - v * 𝚽_k(ζ) 
    #            - ∑_i ( ζ^{2^i} * 𝚽_{k-i-1}(ζ^{2^{i+1}}) - u_i * 𝚽_{k-i}(ζ^{2^i}) ) * q_i(X)
    #          = f(X) - v_𝚽 - ∑_i c_i * q_i(X)
    
    phi_uni_at_zeta_mul_v = periodic_poly(k, 1).evaluate(zeta) * v
    if debug > 1:
        print(f"V> phi_uni_at_zeta_mul_v={phi_uni_at_zeta_mul_v}")

    r_cm = f_cm - kzg10.commit([phi_uni_at_zeta_mul_v])
    for i in range(k):
        c_i = zeta^(pow_2(i)) * periodic_poly(k-i-1, pow_2(i+1)).evaluate(zeta) \
                   - point[i] * periodic_poly(k-i, pow_2(i)).evaluate(zeta)
        r_cm -= c_i * q_cm_vec[i]

    # if debug > 1:
    #     print("P> r_uni=", r_uni)
    # if debug > 0:
    #     assert r_uni.evaluate(zeta) == 0, f"Evaluation does not match, {r_uni.evaluate(zeta)}!=0"
    #     print(f"P> 👀 r(zeta={zeta}) == 0 ✅")

    # compute h(X) = q_hat(X) - ∑_i ( beta^{i} * X^{2^k - 2^i} * q_i(X) ) 
    #
    #    h_ζ(X) = q_hat(X) - ∑_i ( beta^{i} * ζ^{2^k - 2^i} * q_i(X) )

    h_cm = q_hat_cm
    for i in range(k):
        e_i = (beta**i) * (zeta**(pow_2(k) - pow_2(i)))
        h_cm -= e_i * q_cm_vec[i]

    if debug > 1:
        print(f"V> h_cm={h_cm}")
    # if debug > 0:
    #     assert h_uni.evaluate(zeta) == 0, f"Evaluation does not match, {h_uni.evaluate(zeta)}!=0"
    #     print(f"V> 👀 h(zeta={zeta}) == 0 ✅")
    
    alpha = transcript.squeeze()
    print(f"V> ◀️◀️ alpha={alpha}")

    a_cm = h_cm + r_cm * alpha

    # if debug > 0:
    #     assert a_uni.evaluate(zeta) == 0, f"Evaluation does not match, {a_uni.evaluate(zeta)}!=0"
    #     print(f"V> 👀 a(zeta={zeta}) == 0 ✅")

    checked = kzg10.verify_eval_and_degree(a_cm, eval_deg_arg, zeta, 0, kzg10.max_degree - pow_2(k))
    print("V> 👀  a(zeta) == 0 ✅" if checked else "👀  a(zeta) == 0 ❌")
    
    return checked


In [33]:
transcript = ProofTranscript(Fp)

f_uni = UniPolynomial([1, 4, 2, 6])
f_cm = kzg10.commit(f_uni.coeffs)

prf = prove_zeromorph_opt(f_cm, [1, 4, 2, 6], 2, [3, 3], 22, transcript.clone(), debug=2)
verify_zeromorph_opt(f_cm, prf, 2, [3, 3], 22, transcript.clone(), debug=2)

P> f=[1, 4, 2, 6], point=[3, 3], v=22
quotients=[MLEPolynomial([6], 0), MLEPolynomial([1, 2], 1)], rem=22
P> ▶️▶️ C=Commitment(6*X0^3 + 2*X0^2 + 4*X0 + 1)
P> ▶️▶️ point=[3, 3]
P> ▶️▶️ v=22
P> ▶️▶️ q_cm=Commitment(6)
P> ▶️▶️ q_cm=Commitment(2*X0 + 1)
P> f_cm=Commitment(6*X0^3 + 2*X0^2 + 4*X0 + 1), q_cm_vec=[Commitment(6), Commitment(2*X0 + 1)], q_uni_vec=[6, 1 + 2x]
P> ◀️◀️ beta=51
P> coeffs=[0, 0, 0, 0, 1], coeffs[pow_2(i)]=0
P> x_deg_2_to_i_uni[0]=x^3
P> coeffs=[0, 0, 0, 0, 1], coeffs[pow_2(i)]=0
P> x_deg_2_to_i_uni[1]=x^2
P> q_hat_uni=51x^2 + 108x^3
P> ▶️▶️ q_hat_cm=Commitment(-85*X0^3 + 51*X0^2)
P> ◀️◀️ zeta=36
P> f_uni=1 + 4x + 2x^2 + 6x^3, v=22, phi_uni_at_zeta=125
P> r_uni= 57 + 176x + 2x^2 + 6x^3
P> 👀 r(zeta=36) == 0 ✅
P> h_uni= 17 + 13x + 51x^2 + 108x^3
P> 👀 h(zeta=36) == 0 ✅
P> ◀️◀️ alpha=168
P> 👀 a(zeta=36) == 0 ✅
V> ⏬ f_cm=Commitment(6*X0^3 + 2*X0^2 + 4*X0 + 1)
V> ⏬ point=[3, 3]
V> ⏬ v=22
V> ▶️▶️ q_cm=Commitment(6)
V> ▶️▶️ q_cm=Commitment(2*X0 + 1)
V> ◀️◀️ beta=51
V> ▶️▶️ q_

True

In [36]:
kzg10.h * Fp(3) * Fp(4)

12